In [161]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import xml.etree.ElementTree as ET
import pytesseract as pytess
import sys
import skimage.io as skio
from skimage.transform import rescale, resize
# from preprocessing import preprocess
import os

In [162]:
# from PIL import Image
# image_file = "../forms/form01.jpg"
# im = Image.open(image_file)
# print im.size 

In [163]:
template_file = 'template/template_1.xml'

In [164]:
dumps = list()
annotations = template_file
in_file = open(template_file)
tree = ET.parse(in_file)
root = tree.getroot()
jpg = annotations.split('.')[0] + '.jpg'
imsize = root.find('size')
w = int(imsize.find('width').text)
h = int(imsize.find('height').text)

all = list()

for obj in root.iter('object'):
    current = list()
    name = obj.find('name').text
    folder = obj.find('folder').text
    xmlbox = obj.find('bndbox')
    xn = int(float(xmlbox.find('xmin').text))
    xx = int(float(xmlbox.find('xmax').text))
    yn = int(float(xmlbox.find('ymin').text))
    yx = int(float(xmlbox.find('ymax').text))
    current += [jpg,w,h,name,folder,xn,yn,xx,yx]
    all += [current]
    
in_file.close()

In [165]:
def preprocess(img_grey):
    (thresh, img) = cv2.threshold(img_grey, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    # Taking a matrix of size 5 as the kernel
    # The first parameter is the original image,
    # kernel is the matrix with which image is
    # convolved and third parameter is the number
    # of iterations, which will determine how much
    # you want to erode/dilate a given image.
    kernel = np.ones((2,2), np.uint8)
    img = cv2.erode(img, kernel, iterations=1)

    img = cv2.bitwise_not(img)
    cv2.imwrite('temp/bitwiseNot.jpg',img)
    th2 = cv2.adaptiveThreshold(img,255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,-2)
    horizontal = th2
    #vertical = th2
    rows,cols = horizontal.shape
    horizontalsize = int(cols / 45)
    horizontalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (horizontalsize,1))
    horizontal = cv2.erode(horizontal, horizontalStructure, (-1, -1))
    horizontal = cv2.dilate(horizontal, horizontalStructure, (-1, -1))
    cv2.imwrite("temp/horizontal.jpg", horizontal)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    #inverse the image, so that lines are black for masking
    horizontal_inv = cv2.bitwise_not(horizontal)
    #perform bitwise_and to mask the lines with provided mask
    masked_img = cv2.bitwise_and(img, img, mask=horizontal_inv)
    #reverse the image back to normal
    masked_img_inv = cv2.bitwise_not(masked_img)
    cv2.imwrite("temp/temp1.jpg", masked_img_inv)

    # Vertical libe removal
    img2 = cv2.imread("temp/temp1.jpg")
    img2 =cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

    img2 = cv2.bitwise_not(img2)
    th3 = cv2.adaptiveThreshold(img2,255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,15,-2)
    vertical = th3
    verticalsize = int(rows / 70)
    verticalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (1, verticalsize))
    vertical = cv2.erode(vertical, verticalStructure, (-1, -1))
    vertical = cv2.dilate(vertical, verticalStructure, (-1, -1))
    cv2.imwrite("temp/vertical.jpg", vertical)

    vertical_inv = cv2.bitwise_not(vertical)
    #perform bitwise_and to mask the lines with provided mask
    masked_img2 = cv2.bitwise_and(img2, img2, mask=vertical_inv)
    #reverse the image back to normal
    masked_img_inv2 = cv2.bitwise_not(masked_img2)
    #cv2.imshow("vertical_bitwise_not", vertical)
    cv2.imwrite("temp/temp2.jpg", masked_img_inv2)

    #Remove spots from the final output
    image = cv2.imread('temp/temp2.jpg')
    # Denoising spots
    #image = cv2.fastNlMeansDenoisingColored(image,None,10,10,7,30)
    img_bw = 255*(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) > 5).astype('uint8')
    se1 = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    se2 = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
    img_mask = cv2.morphologyEx(img_bw, cv2.MORPH_CLOSE, se1)
    img_mask = cv2.morphologyEx(img_mask, cv2.MORPH_OPEN, se2)

    img_mask = np.dstack([img_mask, img_mask, img_mask]) / 255
    output = image * img_mask
    cv2.imwrite('forms/output.png', output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

In [166]:
data = pd.DataFrame(all, columns=['path','width','height','label','folder','xmin','ymin','xmax','ymax'])

input_image = cv2.imread('forms/form01.jpg')

input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
preprocess(input_image)
img = cv2.imread('forms/output.png')

In [176]:
for i,row in data.iterrows():
    x1 = row['xmin']
    y1 = row['ymin']
    x2 = row['xmax']
    y2 = row['ymax']
    image = img[y1:y2, x1:x2]
    print(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh, bnw = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    path1 = row.folder
    image_path = row.label
    crop_folder_path = 'forms/crop_image/'+ path1
    crop_image_path = crop_folder_path  +'/' + image_path+ '.jpg'
#     print(path)
#     print(crop_folder_path)
#     print(crop_image_path)
    if os.path.exists(crop_folder_path):
        print('Image is segmented into {0} directory'.format(crop_folder_path))
        cv2.imwrite(crop_image_path,bnw)
    if not os.path.exists(crop_folder_path):
        print("Creation of directory {0} successfull".format(crop_folder_path))
        os.mkdir(crop_folder_path)

[[[255 255 255]
  [255 255 255]
  [249 249 249]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[252 252 252]
  [255 255 255]
  [255 255 255]
  ...
  [251 251 251]
  [253 253 253]
  [255 255 255]]

 [[  0   0   0]
  [  0   0   0]
  [255 255 255]
  ...
  [  0   0   0]
  [  2   2   2]
  [254 254 254]]

 ...

 [[250 250 250]
  [255 255 255]
  [255 255 255]
  ...
  [252 252 252]
  [255 255 255]
  [251 251 251]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [253 253 253]
  [254 254 254]]

 [[  1   1   1]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [252 252 252]
  [255 255 255]]]
Image is segmented into forms/crop_image/First_Name directory
[[[252 252 252]
  [  3   3   3]
  [  1   1   1]
  ...
  [255 255 255]
  [  0   0   0]
  [  0   0   0]]

 [[255 255 255]
  [251 251 251]
  [255 255 255]
  ...
  [254 254 254]
  [255 255 255]
  [253 253 253]]

 [[255 255 255]
  [255 255 255]
  [254 254 254]
  ...
  [255 255 255]
  [253 253 253]
  [255 255 255

error: OpenCV(4.1.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
